In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report as cr
import pickle

warnings.simplefilter(action = 'ignore', category = FutureWarning)
k_folds = KFold(n_splits = 10)

In [2]:
dataset = pd.read_csv("Hotel_dataset.csv")
reviews = [w for w in dataset["Review"]]

In [3]:
dataset.loc[dataset.Rating == 1, 'Rating'] = 0
dataset.loc[dataset.Rating == 2, 'Rating'] = 0
dataset.loc[dataset.Rating == 3, 'Rating'] = 0
dataset.loc[dataset.Rating == 4, 'Rating'] = 1
dataset.loc[dataset.Rating == 5, 'Rating'] = 1

In [4]:
pickle_in = open("glove_vectors.pickle", "rb")
word_vectors_GloVe = pickle.load(pickle_in)

In [5]:
def getReviewVector(review):
    return np.mean([word for word in review], axis = 0)

In [6]:
x = []
for i in range(len(word_vectors_GloVe)):
    x.append(getReviewVector(word_vectors_GloVe[i]))

In [7]:
y = pd.get_dummies(dataset["Rating"])
y = y.iloc[:,1].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [9]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [10]:
score = accuracy_score(y_test, y_pred)
print(score)
print(cr(y_test, y_pred))

0.8345938033666748
              precision    recall  f1-score   support

           0       0.80      0.52      0.63      1111
           1       0.84      0.95      0.89      2988

    accuracy                           0.83      4099
   macro avg       0.82      0.74      0.76      4099
weighted avg       0.83      0.83      0.82      4099



In [11]:
# 10-fold cross-validation
scores = cross_val_score(rfc, x, y, cv = k_folds)

In [12]:
# 10-fold cross-validation results
print("CV scores: ", scores)
print("Avg CV score: ", scores.mean())

CV scores:  [0.83268293 0.81259151 0.85505124 0.8340654  0.83162518 0.82869693
 0.85212299 0.84821864 0.8477306  0.86383602]
Avg CV score:  0.8406621433417849
